# Analysis of evaporation

In [98]:
import pandas as pd
import requests
import io
import plotly.express as px
import plotly.graph_objects as go

## First import the data

Note: The data is stored on Githubs Large File System (LFS) any you may not have the data locally. The easiest way to check is look at the file size of one of the .parquet files in the evaporation_data directory.

If the file sizes:<br>
<p style="text-indent: 20px;">~ 4 KB then you do not have it locally</p>
<p style="text-indent: 20px;">~ 400 MB then you have it locally</p>

You can use python to fetch the data for this analysi or you can download it through the Github LFS.

NOTE: 36 GB of RAM memory or larger is recommended to run the code.

In [ ]:
# Select if you have the data locally (3-4 seconds) or not (2-3 minutes)
have_data_locally = False

if have_data_locally:
    air_temp_C = pd.read_parquet("evaporation_data/air_temp_C.parquet")
    dew_point_temp_C =  pd.read_parquet("evaporation_data/dew_point_temp_C.parquet")
    solar_GHI_watt_per_m2 =  pd.read_parquet("evaporation_data/solar_GHI_watt_per_m2.parquet")
    wind_speed_m_per_s =  pd.read_parquet("evaporation_data/wind_speed_m_per_s.parquet")
    area_m2_sqkm_dev_percent_lat_long_elevation_m = pd.read_parquet("evaporation_data/area_m2_sqkm_dev_percent_lat_long_elevation_m.parquet").set_index("cluster_id").T
    area_m2_sqkm_dev_percent_lat_long_elevation_m = area_m2_sqkm_dev_percent_lat_long_elevation_m.columns = air_temp_C.columns

else:

    def load_LFS_data_from_Github(file_name):

        # GitHub API URL to get the file information
        API_URL = f'https://api.github.com/repos/IdahoLabResearch/AquaPV/contents/evaporation_data/{file_name}.parquet?ref=main'

        # Fetch the LFS file metadata from GitHub API
        def fetch_lfs_metadata(api_url):
            response = requests.get(api_url)
            response.raise_for_status()
            return response.json()

        # Get the download URL for the file content
        metadata = fetch_lfs_metadata(API_URL)

        # Check if the file is stored using Git LFS
        if metadata.get('download_url'):

            # Fetch the raw file content from the Git LFS URL
            raw_url = metadata['download_url']
            response = requests.get(raw_url)
            response.raise_for_status()
            file_content = response.content

            # Read the Parquet file into a pandas DataFrame
            df = pd.read_parquet(io.BytesIO(file_content))

            return df
        else:
            print(f"The file {file_name} is not stored using Git LFS.")


    air_temp_C = load_LFS_data_from_Github("air_temp_C")
    dew_point_temp_C = load_LFS_data_from_Github("dew_point_temp_C")
    solar_GHI_watt_per_m2 = load_LFS_data_from_Github("solar_GHI_watt_per_m2")
    wind_speed_m_per_s = load_LFS_data_from_Github("wind_speed_m_per_s")
    area_m2_sqkm_dev_percent_lat_long_elevation_m = load_LFS_data_from_Github("area_m2_sqkm_dev_percent_lat_long_elevation_m").set_index("cluster_id").T
    area_m2_sqkm_dev_percent_lat_long_elevation_m = area_m2_sqkm_dev_percent_lat_long_elevation_m.columns = air_temp_C.columns


## Evapration rate equations and time-series results for all reservoirs

This comes from the paper:<br>
Effects of a Floating Photovoltaic System on the Water Evaporation Rate in the Passaúna Reservoir, Brazil<br>
It can be found  <a href="https://www.researchgate.net/publication/363063415_Effects_of_a_Floating_Photovoltaic_System_on_the_Water_Evaporation_Rate_in_the_Passauna_Reservoir_Brazil">here</a>


In [104]:
# For memory reasons you might only want to do one of the models
# Not following sections will be based off this as well
Linacre_1977_model = False
Linacre_1993_model = True


def Linacre_1993_evap_mm_per_time_step(Temp, T_dew, elevation, solar, wind_speed, time_step):
    """
    Linacre 1993 method for calculating evaporation
    Linacre, E.T. Data-sparse estimation of lake evaporation, using a simplified Penman equation. Agric. For. Meteorol. 1993, 64, 237-256.
    
    Temp: Air temperature (C)
    T_dew: Dewpoint temperature (C)
    elevation: Elevation of waterbody (m)
    solar: Solar irradiance (Watts/m^2)
    wind_speed: wind speed at 2m above ground (m/s)
    time_step: Time step of data (minutes)
    """

    F = (1.0 - 8.7e-5 * elevation)

    evap_rate = (0.015 + 0.00042 * Temp + 1e-6 * elevation) * (0.8 * solar - 40 + 2.5 * F * wind_speed * (Temp - T_dew))


    # The equations are for mm/day but we want to be able to use any time step from data
    evap_rate = evap_rate / (24*60) * time_step

    return evap_rate


def Linacre_1977_evap_mm_per_time_step(Temp, T_dew, elevation, latitude, time_step):
    """
    Linacre 1977 method for calculating evaporation
    Linacre, E.T. Data-sparse estimation of lake evaporation, using a simplified Penman equation. Agric. For. Meteorol. 1993, 64, 237-256.

    Temp: Air temperature (C)
    T_dew: Dewpoint temperature (C)
    elevation: Elevation of waterbody (m)
    latitude: Latitude of waterbody (deg)
    time_step: Time step of data (minutes)
    """
    
    evap_rate = ( 700 * (Temp + 0.006 * elevation) / 100 - latitude) + (15 * (Temp - T_dew)) / (80 - Temp)

    # The equations are for mm/day but we want to be able to use any time step from data
    evap_rate = evap_rate / (24*60) * time_step

    return evap_rate


if Linacre_1993_model:
    Linacre_1993_evap_rate_30_min = Linacre_1993_evap_mm_per_time_step(air_temp_C, dew_point_temp_C, area_m2_sqkm_dev_percent_lat_long_elevation_m.loc["elevation_m"], solar_GHI_watt_per_m2, wind_speed_m_per_s, time_step=30).reindex(columns=air_temp_C.columns)
    Linacre_1993_evap_rate_daily = Linacre_1993_evap_rate_30_min.resample("D").sum()
    Linacre_1993_evap_rate_weekly = Linacre_1993_evap_rate_daily.resample("W").sum()
    Linacre_1993_evap_rate_monthly = Linacre_1993_evap_rate_weekly.resample("M").sum()
    Linacre_1993_evap_rate_yearly = Linacre_1993_evap_rate_monthly.resample("Y").sum()

if Linacre_1977_model:
    Linacre_1977_evap_rate_30_min = Linacre_1977_evap_mm_per_time_step(air_temp_C, dew_point_temp_C, area_m2_sqkm_dev_percent_lat_long_elevation_m.loc["elevation_m"], area_m2_sqkm_dev_percent_lat_long_elevation_m.loc["lat"], time_step=30).reindex(columns=air_temp_C.columns)
    Linacre_1977_evap_rate_daily = Linacre_1977_evap_rate_30_min.resample("D").sum()
    Linacre_1977_evap_rate_weekly = Linacre_1977_evap_rate_daily.resample("W").sum()
    Linacre_1977_evap_rate_monthly = Linacre_1977_evap_rate_weekly.resample("M").sum()
    Linacre_1977_evap_rate_yearly = Linacre_1977_evap_rate_monthly.resample("Y").sum()

## Calculate total evaporation and reduced evaporation for all reservoirs

In [105]:
def calc_total_evaporation_gallons(evap_rate, area):
    """
    Calculate the evaporation over the total waterbody
    evap_rate: Rate of evaporation
    area: Area of waterbody (m^2)

    For units to work we get the mm to m by multiplying 0.001
    Units: (m/t) * m^2 which is m^2/t
    Then m^3 to gallons is multiply 264.172
    Returns gallons/time_step of evap_rate input given
    """

    evaporation_m3 = evap_rate * 0.001 * area

    evpaoration_gallons = evaporation_m3 * 264.172

    return evpaoration_gallons.reindex(columns=evap_rate.columns)


def reduced_evaporation(evaporation, area_percent_FPV, epsilon=0.5):
    """
    This calculates the reduced evaporation due to FPV from
    Effects of a Floating Photovoltaic System on the Water Evaporation Rate in the Passaúna Reservoir, Brazil
    Equation 8 
    
    evaporation: Total evaporation of waterbody (gallons)
    area_percent_FPV: Percent of reservoir covered by FPV (% between 0 and 1)
    epsilon: Evaporation reduction efficiency (% between 0 and 1)
        We use 0.5 as default.

    From: Effects of a Floating Photovoltaic System on the Water Evaporation Rate in the Passaúna Reservoir, Brazil
        The efficiency was estimated from 60.20% to 70.70% for the evaporation reduction by the FPS. 
        Studies have shown that devices such as floating or suspended covers can reduce water evaporation rates by up to 90% [9,13,15,29].
    
    Therefore we assume a 50% reduction which could be considered conservative
    """

    reduced_evaporation = evaporation * area_percent_FPV * epsilon
    
    return reduced_evaporation


area = area_m2_sqkm_dev_percent_lat_long_elevation_m.loc["area_m2"]
area_percent_FPV = area_m2_sqkm_dev_percent_lat_long_elevation_m.loc["dev_percent"]

if Linacre_1977_model:
    Linacre_1977_evaporation_30_min = calc_total_evaporation_gallons(Linacre_1977_evap_rate_30_min, area)
    Linacre_1977_evaporation_daily = calc_total_evaporation_gallons(Linacre_1977_evap_rate_daily, area)
    Linacre_1977_evaporation_weekly = calc_total_evaporation_gallons(Linacre_1977_evap_rate_weekly, area)
    Linacre_1977_evaporation_monthly = calc_total_evaporation_gallons(Linacre_1977_evap_rate_monthly, area)
    Linacre_1977_evaporation_yearly = calc_total_evaporation_gallons(Linacre_1977_evap_rate_yearly, area)

    Linacre_1977_reduced_evaporation_30_min = calc_total_evaporation_gallons(Linacre_1977_evaporation_30_min, area_percent_FPV)
    Linacre_1977_reduced_evaporation_daily = calc_total_evaporation_gallons(Linacre_1977_evaporation_daily, area_percent_FPV)
    Linacre_1977_reduced_evaporation_weekly = calc_total_evaporation_gallons(Linacre_1977_evaporation_weekly, area_percent_FPV)
    Linacre_1977_reduced_evaporation_monthly = calc_total_evaporation_gallons(Linacre_1977_evaporation_monthly, area_percent_FPV)
    Linacre_1977_reduced_evaporation_yearly = calc_total_evaporation_gallons(Linacre_1977_evaporation_yearly, area_percent_FPV)

if Linacre_1993_model:
    Linacre_1993_evaporation_30_min = calc_total_evaporation_gallons(Linacre_1993_evap_rate_30_min, area)
    Linacre_1993_evaporation_daily = calc_total_evaporation_gallons(Linacre_1993_evap_rate_daily, area)
    Linacre_1993_evaporation_weekly = calc_total_evaporation_gallons(Linacre_1993_evap_rate_weekly, area)
    Linacre_1993_evaporation_monthly = calc_total_evaporation_gallons(Linacre_1993_evap_rate_monthly, area)
    Linacre_1993_evaporation_yearly = calc_total_evaporation_gallons(Linacre_1993_evap_rate_yearly, area)

    Linacre_1993_reduced_evaporation_30_min  = calc_total_evaporation_gallons(Linacre_1993_evaporation_30_min, area_percent_FPV)
    Linacre_1993_reduced_evaporation_daily   = calc_total_evaporation_gallons(Linacre_1993_evaporation_daily, area_percent_FPV)
    Linacre_1993_reduced_evaporation_weekly  = calc_total_evaporation_gallons(Linacre_1993_evaporation_weekly, area_percent_FPV)
    Linacre_1993_reduced_evaporation_monthly = calc_total_evaporation_gallons(Linacre_1993_evaporation_monthly, area_percent_FPV)
    Linacre_1993_reduced_evaporation_yearly  = calc_total_evaporation_gallons(Linacre_1993_evaporation_yearly, area_percent_FPV)


## Plot the results of select reservoir

In [122]:
# Select a column to show results
column_id = 10
reservoir_area = area_m2_sqkm_dev_percent_lat_long_elevation_m.iloc[:, column_id]["area_sqkm"]
FPV_dev_percent = area_m2_sqkm_dev_percent_lat_long_elevation_m.iloc[:, column_id]["dev_percent"]

if Linacre_1977_model:
    df = pd.DataFrame(
        {
            "Total Evaporation": Linacre_1977_evaporation_daily.iloc[:, column_id],
            "Reduced Evaporation": Linacre_1977_reduced_evaporation_daily.iloc[:, column_id],
        }
    )
    fig = px.line(df)
    fig.update_layout(title=dict(text=f"Evaporation per Day in Gallons<br>Reservoir Area {reservoir_area:.1f} km<sup>2</sup> with {FPV_dev_percent*100:.1f}% FPV covered", xanchor='center', x=0.5, font=dict(size=20)))
    fig.show()

if Linacre_1993_model:
    df = pd.DataFrame(
        {
            "Total Evaporation": Linacre_1993_evaporation_daily.iloc[:, column_id],
            "Reduced Evaporation": Linacre_1993_reduced_evaporation_daily.iloc[:, column_id],
        }
    )
    fig = px.line(df)
    fig.update_layout(title=dict(text=f"Evaporation per Day in Gallons<br>Reservoir Area {reservoir_area:.1f} km<sup>2</sup> with {FPV_dev_percent*100:.1f}% FPV covered", xanchor='center', x=0.5, font=dict(size=20)))
    fig.show()
